In [1]:
import sys
import os
sys.path.append(os.getcwd() + "/..")
from ws.scraper import Scraper
from selenium import webdriver
from ws.constants import *
import json
import pandas as pd
import numpy as np

In [2]:
LEAGUES

{'Brasileirao': {'countryid': 31, 'leagueid': 95},
 'Bundesliga': {'countryid': 81, 'leagueid': 3},
 'Chinese Super League': {'countryid': 45, 'leagueid': 162},
 'Eredivisie': {'countryid': 155, 'leagueid': 13},
 'FIFA World Cup': {'countryid': 247, 'leagueid': 36},
 'La Liga': {'countryid': 206, 'leagueid': 4},
 'Liga NOS': {'countryid': 177, 'leagueid': 21},
 'Ligue 1': {'countryid': 74, 'leagueid': 22},
 'Major League Soccer': {'countryid': 233, 'leagueid': 85},
 'Premier League': {'countryid': 252, 'leagueid': 2},
 'Primera Argentina': {'countryid': 11, 'leagueid': 68},
 'Russian Premier League': {'countryid': 182, 'leagueid': 77},
 'Serie A': {'countryid': 108, 'leagueid': 5},
 'Super Lig': {'countryid': 225, 'leagueid': 17},
 'UEFA Champions League': {'countryid': 250, 'leagueid': 12},
 'UEFA Europa League': {'countryid': 250, 'leagueid': 30}}

In [3]:
from ws.loader import League, Match, MatchLoader

le = League("Primera Argentina")
le.numberMatches()
print(le.season)

2018-2019


In [4]:
lista = le.matcheslists[le.season]

In [5]:
m = le.importMatch(lista[0])

In [6]:
print(m)

In [7]:
for v in DB.collection_names():
    
    DB[v].remove()

/Users/cgnal/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
load = MatchLoader(DB,m)

In [9]:
load.teams().players().matches().events()

In [10]:
DB["teams"].find_one()

{'_id': ObjectId('5b97cee37425c342464403ca'),
 'teamId': 889,
 'teamName': 'Boca Juniors'}

In [16]:
res = DB["events"].find({"qualifiers.type":"EndX"})

In [18]:
res[0]

{'_id': ObjectId('5b97cee37425c3424644041d'),
 'endX': 50.9,
 'endY': 52.1,
 'eventId': 3,
 'expandedMinute': 0,
 'id': 2095858340.0,
 'isTouch': True,
 'minute': 0,
 'outcomeType': {'displayName': 'Successful', 'value': 1},
 'period': {'displayName': 'FirstHalf', 'value': 1},
 'playerId': 92967,
 'qualifiers': [{'type': {'displayName': 'PassEndY', 'value': 141},
   'value': '52.1'},
  {'type': {'displayName': 'Length', 'value': 212}, 'value': '1.7'},
  {'type': {'displayName': 'Zone', 'value': 56}, 'value': 'Center'},
  {'type': {'displayName': 'Angle', 'value': 213}, 'value': '0.9'},
  {'type': {'displayName': 'PassEndX', 'value': 140}, 'value': '50.9'}],
 'satisfiedEventsTypes': [90, 118, 116, 29, 35, 36, 215, 217],
 'second': 1,
 'teamId': 893,
 'type': {'displayName': 'Pass', 'value': 1},
 'wsEventId': '13144132095858340.0',
 'wsMatchId': 1314413,
 'x': 49.9,
 'y': 50.1}

In [30]:
DB["events"].find({"type.displayName":"Pass"})[1]["qualifiers"]

[{'type': {'displayName': 'PassEndY', 'value': 141}, 'value': '56.8'},
 {'type': {'displayName': 'Zone', 'value': 56}, 'value': 'Back'},
 {'type': {'displayName': 'Angle', 'value': 213}, 'value': '2.9'},
 {'type': {'displayName': 'PassEndX', 'value': 140}, 'value': '38.5'},
 {'type': {'displayName': 'Length', 'value': 212}, 'value': '13.4'}]

In [11]:
DB["matches"].find_one({""})

{'_id': ObjectId('5b97cee37425c34246440416'),
 'awayId': 893,
 'awayManager': 'Juan Pablo Vojvoda',
 'awayName': 'Talleres',
 'date': '2018-08-12T15:00:00',
 'homeId': 889,
 'homeManager': 'Guillermo Barros Schelotto',
 'homeName': 'Boca Juniors',
 'league': 'Primera Argentina',
 'referee': {'firstName': 'Facundo',
  'hasParticipatedMatches': False,
  'lastName': 'Tello',
  'name': 'Facundo Tello',
  'officialId': 1267},
 'season': '2018-2019',
 'venue': 'Alberto José Armando (La Bombonera)',
 'wsMatchId': 1314413}

In [5]:
class Match():
    
    def __init__(self,matchid,league,season,jsonpath=JSONPATH):
        
        self.league = league
        self.season = season
        self.leaguename = self.league.lower().replace(" ","_")
        self.matchid = matchid
        self.jsonpath = jsonpath
        self.matchpath = jsonpath+self.leaguename+"/"+self.season+"/"+str(self.matchid)+".json"
        self.match = json.load(open(self.matchpath,"r",encoding="utf-8"),encoding="utf-8")
        
        self.__matchInfo__()
        self.__events__()
        self.__players__()
        self.__teams__()
        
    def __matchInfo__(self):
        
        matches_dict = {}
        
        try:
            matches_dict["wsmatchid"] = self.matchid
        except KeyError:
            matches_dict["wsmatchid"] = np.nan
        try:
            matches_dict["league"] = self.league
        except KeyError:
            matches_dict["league"] = np.nan
        try:
            matches_dict["season"] = self.season
        except KeyError:
            matches_dict["season"] = np.nan
        try:
            matches_dict["date"] = pd.to_datetime(self.match["startTime"].replace("T"," "))
        except KeyError:
            matches_dict["date"] = np.nan
        try:
            matches_dict["hometeamid"] = self.match["home"]["teamId"]
        except KeyError:
            matches_dict["hometeamid"] = np.nan
        try:
            matches_dict["awayteamid"] = self.match["away"]["teamId"]
        except KeyError:
            matches_dict["awayteamid"] = np.nan
        try:
            matches_dict["hometeamname"] = self.match["home"]["name"]
        except KeyError:
            matches_dict["hometeamname"] = np.nan
        try:
            matches_dict["awayteamname"] = self.match["away"]["name"]
        except KeyError:
            matches_dict["awayteamname"] = np.nan
        try:
            matches_dict["homescore"] = self.match["score"].split(" : ")[0]
        except KeyError:
            matches_dict["homescore"] = np.nan
        try:
            matches_dict["awayscore"] = self.match["score"].split(" : ")[1]
        except KeyError:
            matches_dict["awayscore"] = np.nan
        try:
            if self.match["pkScore"] == '':
                matches_dict["homepkscore"] = np.nan
                matches_dict["awaypkscore"] = np.nan
            else:
                matches_dict["homepkscore"] = self.match["pkScore"].split(" : ")[0]
                matches_dict["awaypkscore"] = self.match["pkScore"].split(" : ")[1]
        except KeyError:
            matches_dict["homepkscore"] = np.nan
            matches_dict["awaypkscore"] = np.nan
        try:
            matches_dict["referee"] = self.match["referee"]["name"]
        except KeyError:
            matches_dict["referee"] = np.nan
        try:
            matches_dict["managerhome"] = self.match["home"]["managerName"]
        except KeyError:
            matches_dict["managerhome"] = np.nan
        try:
            matches_dict["manageraway"] = self.match["away"]["managerName"]
        except KeyError:
            matches_dict["manageraway"] = np.nan
        try:
            matches_dict["attendance"] = self.match["attendance"]
        except KeyError:
            matches_dict["attendance"] = np.nan
        try:
            matches_dict["venuename"] = self.match["venueName"]
        except KeyError:
            matches_dict["venuename"] = np.nan
        
        column_order = ["wsmatchid","league","season","date","hometeamid",
                        "awayteamid","hometeamname","awayteamname","homescore",
                        "awayscore","homepkscore","awaypkscore","referee",
                        "managerhome","manageraway","attendance","venuename"]
        
        self.matchinfo = pd.Series(matches_dict)[column_order]
        
        #return self.matchinfo
    
    def __events__(self):
            
        events = self.match["events"]
        event_list=[]
        qual_list=[]

        for event in events:
            events_dict = {}
            try:
                events_dict["keyid"] = str(self.matchid)+str(event["id"])
            except KeyError:
                events_dict["keyid"] = np.nan
            try:
                events_dict["wsmatchid"] = self.matchid
            except KeyError:
                events_dict["wsmatchid"] = np.nan
            try:
                events_dict["wseventid"] = str(event["id"])
            except KeyError:
                events_dict["wseventid"] = np.nan
            try:
                events_dict["matcheventid"] = event["eventId"]
            except KeyError:
                events_dict["matcheventid"] = np.nan
            try:
                events_dict["minute"] = event["minute"]
            except KeyError:
                events_dict["minute"] = np.nan
            try:
                events_dict["second"] = event["second"]
            except KeyError:
                events_dict["second"] = np.nan
            try:
                events_dict["expandedminute"] = event["expandedMinute"]
            except KeyError:
                events_dict["expandedminute"] = np.nan
            try:
                events_dict["teamid"] = event["teamId"]
            except KeyError:
                events_dict["teamid"] = np.nan
            try:
                events_dict["playerid"] = event["playerId"]
            except KeyError:
                events_dict["playerid"] = np.nan
            try:
                events_dict["period"] = event["period"]["displayName"]
            except KeyError:
                events_dict["period"] = np.nan
            try:
                events_dict["typeid"] = event["type"]["value"]
            except KeyError:
                events_dict["typeid"] = np.nan
            try:
                events_dict["type"] = event["type"]["displayName"]
            except KeyError:
                events_dict["type"] = np.nan
            try:
                events_dict["outcometype"] = event["outcomeType"]["displayName"]
            except KeyError:
                events_dict["outcometype"] = np.nan
            try:
                events_dict["x"] = event["x"]
            except KeyError:
                events_dict["x"] = np.nan
            try:
                events_dict["y"] = event["y"]
            except KeyError:
                events_dict["y"] = np.nan

            event_list.append(events_dict)

            qualifiers = event["qualifiers"]
            
            for qualifier in qualifiers:
                    qual_dict = {}
                    try:
                        qual_dict["keyid"] = events_dict["keyid"]
                    except KeyError:
                        qual_dict["keyid"] = np.nan
                    try:
                        qual_dict["wsmatchid"] = self.matchid
                    except KeyError:
                        qual_dict["wsmatchid"] = np.nan
                    try:
                        qual_dict["wseventid"] = events_dict["wseventid"]
                    except KeyError:
                        qual_dict["wseventid"] = np.nan
                    try:
                        qual_dict["matcheventid"] = events_dict["matcheventid"]
                    except KeyError:
                        qual_dict["matcheventid"] = np.nan
                    try:
                        qual_dict["qualid"] = qualifier["type"]["value"]
                    except KeyError:
                        qual_dict["qualid"] = np.nan
                    try:
                        qual_dict["qualname"] = qualifier["type"]["displayName"]
                    except KeyError:
                        qual_dict["qualname"] = np.nan
                    try:
                        try:
                            value = float(qualifier["value"])
                        except ValueError:
                            value = qualifier["value"]
                        if type(value) == str:
                            qual_dict["qualvaluetext"] = value
                            qual_dict["qualvalue"] = np.nan
                        else:
                            qual_dict["qualvaluetext"] = np.nan
                            qual_dict["qualvalue"] = value    
                    except KeyError:
                        qual_dict["qualvalue"] = 1
                        qual_dict["qualvaluetext"] = np.nan
                    qual_list.append(qual_dict)
    
        self.events = pd.DataFrame(event_list)[["keyid","wsmatchid","wseventid","matcheventid","minute","second","expandedminute","teamid",
                                   "playerid","period","typeid","type","outcometype","x","y"]].set_index("keyid")
        
        self.qualifiers = pd.DataFrame(qual_list)[["keyid","wsmatchid","wseventid","matcheventid","qualid","qualname","qualvalue","qualvaluetext"]].set_index("keyid")
        
        #return self.events, self.qualifiers
        
        
        
    def __players__(self):
        
        dic = [{"playerid":k,"playername":v} for k,v in self.match["playerIdNameDictionary"].items()]
        self.players = pd.DataFrame(dic,index=range(len(dic))).set_index("playerid")
        
        #return self.players
        
    def __teams__(self):
        
        d = {"home":{"teamid":self.matchinfo["hometeamid"],
                "teamname":self.matchinfo["hometeamname"]},
         
        "away":{"teamid":self.matchinfo["awayteamid"],
                "teamname":self.matchinfo["awayteamname"]}}
        
        self.teams = pd.DataFrame(d).T
        
        #return self.teams
        

In [7]:
m = le.importMatch(le.matcheslists["2017-2018"][12])

In [15]:
pd.unique(m.events["period"])

array(['FirstHalf', 'SecondHalf', 'PostGame', 'PreMatch'], dtype=object)

In [8]:
m.match["events"][3]

{'endX': 38.9,
 'endY': 49.9,
 'eventId': 4,
 'expandedMinute': 0,
 'id': 1807661006.0,
 'isTouch': True,
 'minute': 0,
 'outcomeType': {'displayName': 'Successful', 'value': 1},
 'period': {'displayName': 'FirstHalf', 'value': 1},
 'playerId': 14546,
 'qualifiers': [{'type': {'displayName': 'PassEndX', 'value': 140},
   'value': '38.9'},
  {'type': {'displayName': 'Zone', 'value': 56}, 'value': 'Back'},
  {'type': {'displayName': 'Angle', 'value': 213}, 'value': '3.1'},
  {'type': {'displayName': 'PassEndY', 'value': 141}, 'value': '49.9'},
  {'type': {'displayName': 'Length', 'value': 212}, 'value': '11.0'}],
 'satisfiedEventsTypes': [90, 116, 29, 34, 36, 215, 217],
 'second': 2,
 'teamId': 903,
 'type': {'displayName': 'Pass', 'value': 1},
 'x': 49.4,
 'y': 49.1}

In [10]:
import datetime

In [11]:
m.events

wsmatchid     wseventid  matcheventid  minute  second  \
keyid                                                                       
12314211588429394.0   1231421  1588429394.0           2.0       0     0.0   
12314211437274719.0   1231421  1437274719.0           2.0       0     0.0   
12314212061939547.0   1231421  2061939547.0           5.0       0     1.0   
12314211146880972.0   1231421  1146880972.0           6.0       0     2.0   
12314211581883914.0   1231421  1581883914.0           7.0       0     5.0   
12314211346966372.0   1231421  1346966372.0           8.0       0     8.0   
12314211322744539.0   1231421  1322744539.0           9.0       0    10.0   
12314211617717134.0   1231421  1617717134.0          10.0       0    12.0   
12314211318024005.0   1231421  1318024005.0         846.0       0    13.0   
12314211507554853.0   1231421  1507554853.0          11.0       0    18.0   
12314211816477177.0   1231421  1816477177.0          12.0       0    20.0   
12314211890381570.0   1231421  1890381570.0          13.0       0    21.0   
12314211772583178.0   1231421  1772583178.0           3.0       0    22.0   
12314211626001093.0   1231421  1626001093.0           4.0       0    29.0   
12314211674986138.0   1231421  1674986138.0           5.0       0    33.0   
12314211288681801.0   1231421  1288681801.0           6.0       0    38.0   
12314211657750550.0   1231421  1657750550.0           7.0       0    41.0   
12314212053355009.0   1231421  2053355009.0           8.0       0    45.0   
12314211360502699.0   1231421  1360502699.0          14.0       0    49.0   
12314211827889219.0   1231421  1827889219.0          15.0       0    51.0   
12314211365689440.0   1231421  1365689440.0          16.0       0    57.0   
12314211437955497.0   1231421  1437955497.0          17.0       1     0.0   
12314211910135283.0   1231421  1910135283.0           9.0       1     0.0   
12314211681705951.0   1231421  1681705951.0          10.0       1     1.0   
12314211568613364.0   1231421  1568613364.0          11.0       1     2.0   
12314211549512917.0   1231421  1549512917.0          13.0       1     5.0   
12314211691324615.0   1231421  1691324615.0          18.0       1     5.0   
12314211729894995.0   1231421  1729894995.0          14.0       1    38.0   
12314211908623014.0   1231421  1908623014.0          15.0       1    41.0   
12314211739978694.0   1231421  1739978694.0          19.0       1    41.0   
...                       ...           ...           ...     ...     ...   
12314211587673046.0   1231421  1587673046.0         734.0      91    10.0   
12314211750956911.0   1231421  1750956911.0         735.0      91    14.0   
12314211780557644.0   1231421  1780557644.0         821.0      91    17.0   
12314211176916963.0   1231421  1176916963.0         822.0      91    18.0   
12314211688815377.0   1231421  1688815377.0         823.0      91    20.0   
12314211490062763.0   1231421  1490062763.0         824.0      91    23.0   
12314211130015288.0   1231421  1130015288.0         825.0      91    30.0   
12314211418378459.0   1231421  1418378459.0         737.0      91    57.0   
12314211532057274.0   1231421  1532057274.0         828.0      92    12.0   
12314211236562706.0   1231421  1236562706.0         829.0      92    16.0   
12314211301539882.0   1231421  1301539882.0         830.0      92    19.0   
12314211965591827.0   1231421  1965591827.0         831.0      92    26.0   
12314211438641976.0   1231421  1438641976.0         832.0      92    32.0   
12314211689846323.0   1231421  1689846323.0         739.0      92    34.0   
12314211510527178.0   1231421  1510527178.0         833.0      92    34.0   
12314211652955701.0   1231421  1652955701.0         834.0      92    35.0   
12314211655350354.0   1231421  1655350354.0         835.0      92    39.0   
12314212055302228.0   1231421  2055302228.0         836.0      92    39.0   
12314211122197296.0   1231421  1122197296.0         837.0      92    41.0   
1231421165249104

In [33]:
datetime.timedelta(minutes=1,seconds=60).total_seconds()

120.0

In [16]:
m.events.apply(lambda x: datetime.timedelta(seconds=x.second))

AttributeError: ("'Series' object has no attribute 'second'", 'occurred at index wsmatchid')

In [26]:
np.unique([i["type"]["displayName"] for i in m.match["events"]])

array(['Aerial', 'BallRecovery', 'BallTouch', 'BlockedPass', 'Card',
       'Challenge', 'Clearance', 'CornerAwarded', 'Dispossessed', 'End',
       'Error', 'FormationChange', 'FormationSet', 'Foul', 'Goal',
       'Interception', 'KeeperPickup', 'MissedShots', 'OffsideGiven',
       'OffsidePass', 'OffsideProvoked', 'Pass', 'PenaltyFaced', 'Punch',
       'Save', 'SavedShot', 'ShotOnPost', 'Start', 'SubstitutionOff',
       'SubstitutionOn', 'Tackle', 'TakeOn'],
      dtype='<U15')

In [6]:
from pymongo import MongoClient

In [7]:
c = MongoClient()

In [13]:
c.drop_database("football")

In [8]:
c.database_names()

['admin', 'local']

In [13]:
db = c["football"]

In [17]:
db["teams"]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'football'), 'teams')

In [19]:
teams = db["teams"]

In [31]:
for matchid in le.matcheslists["2017-2018"]:
    
    match = le.getMatch(matchid)
    
    team1 = {"teamId":match.match["home"]["teamId"],
            "teamName":match.match["home"]["name"],
            }
    team2 = {"teamId":match.match["away"]["teamId"],
            "teamName":match.match["away"]["name"],
            }
    
    if teams.count({"teamId":team1["teamId"]}) == 0:
        teams.insert(team1)
        print("team inserted")

/Users/cgnal/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted
team inserted


In [45]:
for matchid in le.matcheslists["2017-2018"]:
    
    match = le.getMatch(matchid).match

    teams.update_one({'teamId': match['home']['teamId']}, 
                     {'$setOnInsert': {'teamName': match['home']['name']}}, upsert=True)
    teams.update_one({'teamId': match['away']['teamId']}, 
                     {'$setOnInsert': {'teamName': match['away']['name']}}, upsert=True)
    
    for k,v in match["playerIdNameDictionary"].items():
        
         db["players"].update_one({"playerId":int(k)},
                                 {'$setOnInsert': {'playerName': v}}, upsert=True)
        
        

In [52]:
teams.find_one({"teamName":"Boca Juniors"})["teamId"]

889

In [51]:
str(db["players"].find_one()["_id"])

'5b979bdb7425c3424643aa92'

In [48]:
match.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [59]:
match["startTime"]

'2017-09-24T18:05:00'

In [61]:
match["home"].keys()

dict_keys(['teamId', 'formations', 'stats', 'incidentEvents', 'shotZones', 'name', 'countryName', 'players', 'managerName', 'scores', 'field', 'averageAge'])

In [11]:
match["events"][1]

NameError: name 'match' is not defined